# Chi Square Test


https://passel2.unl.edu/view/lesson/9beaa382bf7e/8#:~:text=If%20your%20chi%2Dsquare%20calculated,to%20reject%22%20your%20null%20hypothesis.

If your chi-square calculated value is greater than the chi-square critical value, then you reject your null hypothesis.
Rejecting hypothesis means we assume something other than chance was at play.

In [1]:
chi_square_value=6.35 # value for 0.01 configence (should only occur in 0.01 of all cases other than by chance)

In [2]:
import pandas as pd
#df = pd.read_parquet('sources/metadata.parquet', engine='pyarrow')
#df_large = pd.read_parquet('sources/metadata-large.parquet', engine='pyarrow')

exact_matches = pd.read_parquet("../results/artists_exact_match_large.parquet", engine='pyarrow')
assert "artists" in exact_matches.columns, f'artists is not in {exact_matches.columns}'
assert "num_artists" in exact_matches.columns, f'num_artists is not in {exact_matches.columns}'

print(f'exact match shapes {exact_matches.shape}')

# read excel_artist_names
import my_utils

excel_artist_names = my_utils.read_lines_as_list("../sources/excel_artists_copy_paste_name.txt")
hundred_artist_names = excel_artist_names[0:100]
ten_artist_names = excel_artist_names[0:10]


artist_mentions = pd.read_parquet('../results/artist_mentions.parquet', engine='pyarrow')

exact match shapes (1819808, 5)


In [3]:
import wordcloud
from collections import defaultdict
import re

def read_stopwords():
    # from https://gist.github.com/sebleier/554280
    return my_utils.read_lines_as_list("stopwords.txt")

def compute_freq(df, col, sw):
    # print(sw)
    freq = defaultdict(lambda: 0)
    for i, row in df.iterrows():
        string = row[col]
        for s in re.split("[, \-!?:]+", string):
            s=s.strip().lower()
            if s not in sw:
                if len(s)>0:
                    freq[s]=freq[s]+1
    return freq



In [4]:
stopwords=read_stopwords()
stopwords.append("greg")
stopwords.append("rutkowski")

freqs = compute_freq(exact_matches, "prompt", stopwords)
print(f'finished')

greg_df = my_utils.exact_match_dataframe(exact_matches, "greg rutkowski")
freqs_greg = compute_freq(greg_df, "prompt", stopwords)
print(f'finished')

finished
finished


In [1]:
import math

print(f'{len(freqs.keys())} items in plain, {len(freqs_greg.keys())} items in greg')

sum_plain = 0
for i in freqs.values():
    sum_plain+=i
    
sum_greg = 0
for i in freqs_greg.values():
    sum_greg+=i

word_chi_values ={}    
for w, freq_greg in freqs_greg.items():
    word_greg = freq_greg
    word_plain = freqs[w]
    others_greg = sum_greg - word_greg
    others_plain = sum_plain - word_plain
    total = sum_greg + sum_plain
    word = word_greg + word_plain
    others = others_greg + others_plain
    expected_word_greg = (word * sum_greg) / total
    expected_word_plain = (word * sum_plain) / total
    expected_others_greg = (others * sum_greg) / total
    expected_others_plain = (others * sum_plain) / total
    word_chi_values[w] = math.pow(word_greg - expected_word_greg,2)/expected_word_greg \
        + math.pow(word_plain - expected_word_plain,2)/expected_word_plain \
        + math.pow(others_greg - expected_others_greg,2)/expected_others_greg\
        + math.pow(others_plain - expected_others_plain,2)/expected_others_plain
    
    
    #print(f'{word_greg} | {word_plain} || {word}')
    #print(f'{others_greg} | {others_plain} || {others}')
    #print(f'{sum_greg} | {sum_plain} || {total}')
    
    #print(f'{expected_word_greg} | {expected_word_plain}')
    #print(f'{expected_others_greg} | {expected_others_plain}')
    #print(f'{w} --> {word_chi_values[w]}')
    
#print(word_chi_values)

chi_values = pd.DataFrame.from_dict(word_chi_values, orient="index", columns = ["chi_values"])
chi_values.index.name = 'word'

chi_values.sort_values("chi_values", axis=0, ascending=False, inplace=True)
chi_values.reset_index(inplace=True)

print(f'amount of words with higher than threshold chi value: {chi_values[chi_values["chi_values"]>=chi_square_value].shape[0]}')

print(chi_values.head(40))


NameError: name 'freqs' is not defined